In [1]:
import pandas as pd
import json
import numpy as np
import pickle


In [2]:
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]


In [3]:
for i in state_names:
  data = json.load(open(i+'.json'))
  if i=='Alaska':
    df=pd.DataFrame(data)
  else:
    df_new=pd.DataFrame(data)
    df = pd.concat((df,df_new))

In [62]:
df.reset_index(drop=True, inplace=True)

In [29]:
df.describe()

,ID,popularity,length,elevation_gain,avg_rating,area_id,country_id,city_id,state_id,verified_map_id,num_reviews,trail_id
count,5.533000e+04,55330.000000,5.533000e+04,55330.000000,55330.000000,4.496200e+04,55330.000000,55330.000000,55330.000000,5.532900e+04,55330.000000,5.533000e+04
mean,1.023438e+07,6.214606,1.222538e+04,348.714915,3.992906,1.013437e+07,313.505765,9156.520549,81.494433,1.880177e+07,43.934195,1.023438e+07
std,1.439038e+05,5.567607,7.894564e+04,1354.604213,0.928263,2.264382e+04,5.152693,17878.568998,586.360491,3.906424e+06,116.058309,1.439038e+05
min,1.000000e+07,0.000000,0.000000e+00,0.000000,0.000000,1.010678e+07,313.000000,3.000000,1.000000,1.325423e+07,0.000000,1.000000e+07
25%,1.003996e+07,2.773525,3.379614e+03,41.757600,4.000000,1.011682e+07,313.000000,2563.000000,6.000000,1.456510e+07,5.000000,1.003996e+07
50%,1.026977e+07,4.673400,6.598294e+03,139.903200,4.000000,1.012061e+07,313.000000,5297.500000,24.000000,1.872130e+07,14.000000,1.026977e+07
75%,1.032874e+07,7.758550,1.287472e+04,409.956000,4.500000,1.016020e+07,313.000000,7858.000000,39.000000,2.216021e+07,39.000000,1.032874e+07
max,1.054926e+07,84.622900,1.153704e+07,179826.818400,5.000000,1.017168e+07,366.000000,151005.000000,6055.000000,2.914324e+07,4447.000000,1.054926e+07


In [49]:
df.columns

Index(['objectID', 'ID', 'slug', 'name', 'popularity', 'type', '_geoloc',
       'length', 'elevation_gain', 'difficulty_rating', 'route_type',
       'visitor_usage', 'avg_rating', 'area_id', 'area_name', 'country_id',
       'country_name', 'city_id', 'city_name', 'state_id', 'state_name',
       'verified_map_id', 'features', 'activities', 'feature_names',
       'activity_names', 'overview', 'num_reviews', 'units', 'area_slug',
       'trail_id', 'city_url', 'park_slug', 'profile_photo_url', 'text',
       'textstring'],
      dtype='object')

In [35]:
df=df[['name', 'popularity', 'length', 'elevation_gain', 'difficulty_rating','avg_rating', 'state_name', 'features', 'activities', 'num_reviews', 'area_name']]

,count
visitor_usage,
2,27137
1,18374
,5073
3,4478
4,268


In [63]:
df['text']=(df['features']+df['activities']).apply(set).apply(list)

In [64]:
df['textstring'] = [' '.join(map(str, l)) for l in df['text']]

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances

def model_tfidf_num(df):

    # scale numerical data (continuous) : global_rank_score missing values
    df_numerical = df[['popularity', 'length',
                                       'elevation_gain','difficulty_rating',
                                       'avg_rating']]
    scaler_num = StandardScaler().fit(df_numerical)
    df_numerical['difficulty_rating'] = df_numerical['difficulty_rating'].astype(float)
    df_num_scaled = scaler_num.transform(df_numerical)

    # vectorize text data
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['textstring']).toarray()

    # concatenate and get similarity
    all_features = np.concatenate([tfidf_matrix,df_num_scaled],axis=1)
    #cosine_sim = cosine_similarity(all_features, all_features)

    return all_features

def get_similar(idx, n):

#idx: target item's index

    # 1. compute distance
    target_feature = all_features[idx].reshape(1, -1)
    print(target_feature.shape, all_features.shape)
    couple_dist = pairwise_distances(all_features,
                                     target_feature, metric='cosine')
    # 2. get similar dataframe: no need to filter out the first
    # because the first won't be the unseen url
    indices = list(
        map(lambda x: x.item(), np.argsort(couple_dist.ravel())))
    # similar_score
    cosine_similarity = 1 - couple_dist[indices].ravel()

    df_sim_all = pd.DataFrame(
        {"tfidf_index": indices, "similar_score": cosine_similarity})

    df_sim = df_sim_all[1:n+1]
    return df[['name','state_name']].iloc[df_sim['tfidf_index']]

In [41]:
all_features=model_tfidf_num(df)

<ipython-input-40-dad5d560cb0e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numerical['difficulty_rating'] = df_numerical['difficulty_rating'].astype(float)


In [ ]:
pickle.dump(df, open('trails.pkl', 'wb'))
pickle.dump(all_features, open('all_features.pkl', 'wb')